# AzureAIAgent Usage Examples

This notebook demonstrates how to use the `AzureAIAgent` with various tools, including Bing grounding, file search, and code interpreter.

In [2]:
import asyncio
import os
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import DefaultAzureCredential
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.agents.azure._azure_ai_agent import AzureAIAgent
import dotenv

# Load environment variables
dotenv.load_dotenv()

True

## Initialize AzureAIAgent

In [3]:
credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential, conn_str=os.getenv("AI_PROJECT_CONNECTION_STRING")
)

## Example 1: Using Bing Grounding Tool

In [18]:
from azure.ai.projects import models

conn = await project_client.connections.get(connection_name=os.getenv("BING_CONNECTION_NAME"))

bing_tool = models.BingGroundingTool(conn.id)
agent_with_bing_grounding = AzureAIAgent(
    name="bing_agent",
    description="An AI assistant with Bing grounding",
    project_client=project_client,
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    tools=bing_tool.definitions,
    metadata={"source": "AzureAIAgent"},
)

async def bing_example():
    result = await agent_with_bing_grounding.on_messages(
        messages=[TextMessage(content="What is Microsoft's annual leave policy?", source="user")],
        cancellation_token=CancellationToken(),
        message_limit=5
    )
    print(result)

await bing_example()

Response(chat_message=TextMessage(source='bing_agent', models_usage=None, metadata={}, content="Microsoft's annual leave policy typically includes a combination of vacation days, sick leave, and other types of leave for their employees. However, the specifics can vary depending on the country, state, and individual employment contracts. Here's a general overview based on their usual practices in the United States:\n\n1. **Vacation Leave**: Microsoft offers a generous vacation policy, often starting at around 15 days (3 weeks) per year for new employees, with increases based on years of service.\n\n2. **Sick Leave**: Employees are provided with separate sick leave allowances.\n\n3. **Holidays**: Microsoft also observes public holidays, giving employees additional days off during the year.\n\n4. **Personal Days**: Some Microsoft employees have access to personal days, which can be used for personal matters.\n\n5. **Parental Leave**: Microsoft offers parental leave, which includes weeks o

## Example 2: Using File Search Tool

In [10]:
agent_with_file_search = AzureAIAgent(
    name="file_search_agent",
    description="An AI assistant with file search capabilities",
    project_client=project_client,
    model="gpt-4o",
    instructions="You are a helpful assistant.",
    tools=["file_search"],
    metadata={"source": "AzureAIAgent"},
)

async def file_search_example():
    await agent_with_file_search.on_upload_for_file_search(
        file_paths=["/workspaces/autogen/python/packages/autogen-core/docs/src/user-guide/core-user-guide/cookbook/data/product_info_1.md"],
        vector_store_name="file_upload_index",
        vector_store_metadata={"source": "AzureAIAgent"},
        cancellation_token=CancellationToken(),
    )
    result = await agent_with_file_search.on_messages(
        messages=[TextMessage(content="Hello, what Contoso products do you know?", source="user")],
        cancellation_token=CancellationToken(),
        message_limit=5
    )
    print(result)

await file_search_example()

Response(chat_message=TextMessage(source='file_search_agent', models_usage=None, metadata={}, content='One of the Contoso products is the "SmartView Glasses" from Contoso Galaxy Innovations. These smart eyewear glasses have features such as an augmented reality interface, a voice-controlled AI assistant, HD video recording with 3D audio, UV protection, blue light filtering, and wireless charging with an extended battery life【4:0†source】.', type='TextMessage'), inner_messages=[])


## Example 3: Using Code Interpreter Tool

In [9]:
agent_with_code_interpreter = AzureAIAgent(
    name="code_interpreter_agent",
    description="An AI assistant with code interpreter capabilities",
    project_client=project_client,
    model="gpt-4o",
    agent_id="asst_LZ0La6Y9LWi2KxAnsrUjRNFw",
    instructions="You are a helpful assistant.",
    tools=["code_interpreter"],
    metadata={"source": "AzureAIAgent"},
)

async def code_interpreter_example():
    await agent_with_code_interpreter.on_upload_for_code_interpreter(
        file_paths="/workspaces/autogen/python/packages/autogen-core/docs/src/user-guide/core-user-guide/cookbook/data/nifty_500_quarterly_results.csv",
        cancellation_token=CancellationToken()
    )
    result = await agent_with_code_interpreter.on_messages(
        messages=[TextMessage(content="Aggregate the number of stocks per industry and give me a markdown table as a result?", source="user")],
        cancellation_token=CancellationToken()
    )
    print(result)

await code_interpreter_example()

Response(chat_message=TextMessage(source='code_interpreter_agent', models_usage=None, metadata={}, content='Here is the markdown table showing the number of stocks per industry:\n\n```markdown\n| Industry                            |   Number of Stocks |\n|:------------------------------------|-------------------:|\n| PHARMACEUTICALS                     |                 33 |\n| BANKS                               |                 28 |\n| AUTO PARTS & EQUIPMENT              |                 20 |\n| IT CONSULTING & SOFTWARE            |                 19 |\n| SPECIALTY CHEMICALS                 |                 18 |\n| FINANCE (INCLUDING NBFCS)           |                 17 |\n| ELECTRIC UTILITIES                  |                 14 |\n| INTERNET SOFTWARE & SERVICES        |                 14 |\n| CEMENT & CEMENT PRODUCTS            |                 13 |\n| REALTY                              |                 11 |\n| IRON & STEEL/INTERM.PRODUCTS        |                  9 |\n